In [1]:
import os
import sys
import re
sys.path.append("../src")

from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from yt_encoder import YTEncoder
from torch.utils.data import Dataset, DataLoader, RandomSampler
from tqdm import tqdm

import torch
import numpy as np

from pathlib import Path

PATH_TO_DATA = Path("../data")
PATH_TO_MODELS = Path("../models")

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
tokenizer = YTEncoder.from_pretrained(str(PATH_TO_MODELS / "yt.model"))
model = GPT2LMHeadModel.from_pretrained(str(PATH_TO_MODELS / "s_gpt_2/")).to(device)

In [4]:
def preprocess(line):
    line = line.replace("І", "I")       # yes, they differ
    line = line.replace("Q", "q")
    line = line.replace("Y", "y")
    line = line.replace("Z", "z")
    line = line.replace("Ъ", "ъ")
    line = line.replace("&", "и")
    line = line.replace("$", "s")
    line = line.replace("\xa0", " ")
    line = line.replace("\x97", " ")
    line = line.replace("\u200b", " ")
    line = line.replace("―", "-")
    line = line.replace("`", "")
    line = re.sub(r"\s+", " ", line)
    return line

In [5]:
wronged = 0
with open(PATH_TO_DATA / "gpt2_dataset.txt") as f:
    for line in f.readlines():
        line = preprocess(line)
        if 1 in tokenizer.encode(line):
            wronged += 1
print(wronged)

0


In [6]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [7]:
def generate_some_text(input_str, text_len = 250):

    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)

    model.eval()
    with torch.no_grad():

        for i in range(text_len):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=10) #Randomly(from the given probability distribution) choose the next word from the top n words
            if next_token_id == 3:
                break
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode([output_list])
        return output_text

In [8]:
class SynopsisDataset(Dataset):
    def __init__(self, dataset_path=(PATH_TO_DATA / "gpt2_dataset.txt")):
        super().__init__()
        self.examples = []
        with open(dataset_path, "r") as f:
            for line in f.readlines():
                line = "СИНОПСИС: " + preprocess(line)
                self.examples.append(tokenizer.encode(line) + [3])

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

In [9]:
EPOCHS = 10
LEARNING_RATE = 3e-5
WARMUP_STEPS = 500

In [10]:
dataset = SynopsisDataset()
sampler = RandomSampler(dataset)
synopsis_loader = DataLoader(dataset, sampler=sampler)

In [11]:
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS,
                                            num_training_steps=EPOCHS*len(dataset))

proc_seq_count = 0
batch_count = 0

for epoch in range(EPOCHS):

    sum_loss = []

    model.train()
    t = tqdm(enumerate(synopsis_loader), total=len(dataset), desc=f"Epoch {epoch}, loss: ")
    for idx, batch in t:

        batch = batch.to(device)
        outputs = model(batch, labels=batch)

        loss = outputs[0]
        loss.backward()
        sum_loss.append(loss.item())

        del outputs, loss, batch
        torch.cuda.empty_cache()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        model.zero_grad()

        t.set_description(f"Epoch {epoch}, loss: {np.mean(sum_loss) :.2}")

    torch.save(model.state_dict(), str(PATH_TO_MODELS / f"gpt2_epoch_{epoch}_autostart.pt"))

    model.eval()
    print("\n" + "="*10+ f"Epoch {epoch}" + "="*10 + "\n")
    print(generate_some_text(" СИНОПСИС: "))


Epoch 9, loss: 2.4: 100%|██████████| 4786/4786 [14:06<00:00,  5.65it/s]



==========Epoch 0==========

СИНОПСИС: В фильме рассказывается о двух друзьях, которые решили вместе совершить путешествие по Южной Корее. Они решили совершить это путешествие вместе с братом по имени Оуэн, чтобы помочь ему найти путь домой, а также сделать его более безопасным от всех опасностей и опасностей.

==========Epoch 1==========

СИНОПСИС: Главный персонаж — агент ФБР, который пытается раскрыть тайну своего прошлого, которое он потерял при загадочных обстоятельствах, совершив головокружительную карьеру.

==========Epoch 2==========

СИНОПСИС: История жизни и карьеры легендарного актера Чарли Чаплина, создав неповторимое представление о жизни и кино: от первых ролей в фильмах до трагической гибели великого актера

==========Epoch 3==========

СИНОПСИС: В центре событий бывший агент спецслужб, ведущий расследование серии убийств. В результате своей операции он узнает страшную тайну, которая навсегда изменит его жизнь. В надежде на то, что сможет раскрыть ее и раскрыть весь мир